<a href="https://colab.research.google.com/github/tekoryu/rough-on-rats/blob/main/notebooks/model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Engenharia de Softwares Inteligentes

## TITANIC CHALLENGE

Desafio, dados originais e detalhes da base disponiveis em:

https://www.kaggle.com/competitions/titanic

In [47]:
import re

import pandas as pd


## Carga do Dataset

Os arquivos do dataset possuem um arquivo de teste, que será ignorado e testado apenas ao final.

In [48]:
# Arquivos do Kaggle
url = "https://raw.githubusercontent.com/tekoryu/rough-on-rats/main/data/titanic/train.csv"
url2 = "https://raw.githubusercontent.com/tekoryu/rough-on-rats/main/data/titanic/test.csv"

# Carga no Pandas
df_treino = pd.read_csv(url, delimiter=',')
df_teste = pd.read_csv(url2, delimiter=',')

# Exibe a primeira linha do dataset
df_treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Tratamento da entrada

Da observação dos dados temos como premissa que a definição de quem foi salvo ou não se deu por motivo de avaliação de atributos (classe econômica, gênero, idade etc).
Vamos buscar reforçar esses atributos a partir dos dados disponíveis.

### Tratamento de Null e em branco

In [49]:
# Lista a quantidade de campos vazios no dataset
df_treino.isna().sum(axis=0).sort_values(ascending=False)
df_teste.isna().sum(axis=0).sort_values(ascending=False)

Cabin          327
Age             86
Fare             1
Name             0
Pclass           0
PassengerId      0
Sex              0
Parch            0
SibSp            0
Ticket           0
Embarked         0
dtype: int64

Como visto, tanto nos arquivos de treino quanto de teste ha muitos vazios em Cabin e Age.

In [50]:
# Decide por eliminar a feature 'Cabin' e preencher os 'em branco' da idade com a mediana. O código será reaproveitado para tratar o arquivo de teste.
def preenche_branco(entrada):
    entrada.drop('Cabin', axis=1, inplace=True)
    entrada = entrada.fillna(entrada['Age'].median())
    return entrada
    
df_teste = preenche_branco(df_teste)

Agora é a hora de extrair o título de cada um deles.

In [74]:
def pega_pronome_tratamento(entrada):
    # A entrada deve ser um dataframe. Pega o Mr. Mrs. e coloca em campo específico
    pronome_tratamento = re.search(r' ([A-Za-z]+)\.',entrada)
    if pronome_tratamento:
        return pronome_tratamento.group(1)
    return ""

df_treino['Pronome'] = df_treino['Name'].apply(pega_pronome_tratamento)


In [76]:
df_treino['Ticket'].unique()

array(['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450',
       '330877', '17463', '349909', '347742', '237736', 'PP 9549',
       '113783', 'A/5. 2151', '347082', '350406', '248706', '382652',
       '244373', '345763', '2649', '239865', '248698', '330923', '113788',
       '347077', '2631', '19950', '330959', '349216', 'PC 17601',
       'PC 17569', '335677', 'C.A. 24579', 'PC 17604', '113789', '2677',
       'A./5. 2152', '345764', '2651', '7546', '11668', '349253',
       'SC/Paris 2123', '330958', 'S.C./A.4. 23567', '370371', '14311',
       '2662', '349237', '3101295', 'A/4. 39886', 'PC 17572', '2926',
       '113509', '19947', 'C.A. 31026', '2697', 'C.A. 34651', 'CA 2144',
       '2669', '113572', '36973', '347088', 'PC 17605', '2661',
       'C.A. 29395', 'S.P. 3464', '3101281', '315151', 'C.A. 33111',
       'S.O.C. 14879', '2680', '1601', '348123', '349208', '374746',
       '248738', '364516', '345767', '345779', '330932', '113059',
       'SO/C 14885', '31012